# Getting started with LangTrace and Snowflake

## 1. Setup LangTrace and Snowflake

* Sign-up for a free account on [LangTrace](https://langtrace.ai/.com "Title").

* Create a Project and generate an API Key

* <span style="color:red"> Note you will also sign up and create an account with [Snowflake](https://www.snowflake.com/ "Snowflake") if you havent done so already </span>


<br/>

## 2. Install packages and setup environment

In [ ]:
pip install langtrace-python-sdk snowflake-connector-python

<br/>

## 3. Imports

In [ ]:

import os
import snowflake.connector

os.environ['LANGTRACE_API_KEY'] = "YOUR_LANGTRACE_API_KEY"
os.environ['PINECONE_API_KEY'] = "YOUR_PINECONE_API_KEY"

In [ ]:
from langtrace_python_sdk import langtrace # Must precede any llm module imports
langtrace.init(api_key = os.environ['LANGTRACE_API_KEY'])


<br/>

## 4. Connect to snowflake Database

In [ ]:
## Replace these with your own credentials.
account = 'account_name'
user = 'user_name'
password = 'password_name'
database = 'db_name'
schema = 'schema_name'


In [ ]:
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    database=database,
    schema=schema,
)

In [ ]:
cur = conn.cursor()

<br/>

## 5. Create some vectors and find the closest distance between them

In [ ]:
cur.execute("CREATE OR REPLACE TABLE vectors (a VECTOR(FLOAT, 3), b VECTOR(FLOAT, 3))")
values = [([1.1, 2.2, 3], [1, 1, 1]), ([1, 2.2, 3], [4, 6, 8])]
for row in values:
        cur.execute(f"""
            INSERT INTO vectors(a, b)
                SELECT {row[0]}::VECTOR(FLOAT,3), {row[1]}::VECTOR(FLOAT,3)
        """)

# Compute the pairwise inner product between columns a and b
cur.execute("SELECT VECTOR_INNER_PRODUCT(a, b) FROM vectors")

In [ ]:
# Find the closest vector to [1,2,3]
cur.execute(f"""
    SELECT a, VECTOR_COSINE_SIMILARITY(a, {[1,2,3]}::VECTOR(FLOAT, 3))
        AS similarity
        FROM vectors
        ORDER BY similarity DESC
        LIMIT 1;
""")
print(cur.fetchall())

<br/>

## You can now view your traces on the Langtrace dashboard

![image info](./assets/image1.png)